In [1]:
import pandas as pd
import os
import glob
import csv

In [2]:
ppi_input_file = 'input/ppi.csv'
seed_input_file = 'input/扩张性心肌病.csv'

disease_name = seed_input_file.split('/')[-1].split('.')[0]

ppi_output_dir = 'graph'
if not os.path.isdir(ppi_output_dir):
    os.mkdir(all_nodes_output)
edge_list_file = ppi_output_dir + '/edge_list.edgelist'
node_list_file = ppi_output_dir + '/node_list.csv'


disease_output_dir = ppi_output_dir+ '/' + disease_name
if not os.path.exists(disease_output_dir):
    os.makedirs(disease_output_dir)

# 输出疾病种子与id的对应关系
disease_seed_list_file = disease_output_dir + '/seed_id_list.csv'
# 输出根据疾病网络得到一步作用剪辑网络（node_id1, node_id2）
disease_edge_list_file = disease_output_dir + '/edges.edgelist'
# 输出一步作用网络得到的节点标签（id, name, label）
disease_node_list_file = disease_output_dir + '/label.csv'

# 数据预处理

In [6]:
# 读取所有nodes的PPI网络
ppi_graph = pd.read_csv('ppi.csv', header=None, index_col=0,  encoding='utf-8')

/home/liuqinyi/anaconda3/envs/bionev/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (3) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


## 对整个PPI网络预处理，得到所有节点的nodelist(id, name), edgelist(id1, id2)，nodelable(id, label)

In [3]:
# 通过将ppi网络中的两列数据进行合并，并去重重建索引，得到存储所有id 和 name 的对应关系表 nodelist
if not os.path.exists(node_list_file):
    all_nodes = pd.concat([ppi_graph[1], ppi_graph[2]]).drop_duplicates().reset_index(drop=True)

    # 写入nodelist
    node_name = pd.Series(all_nodes, name='gene_name')
    node_id = pd.Series(range(len(node_name)), name='id')
    node_list = pd.concat([node_id, node_name], axis=1)
    node_list.to_csv(node_list_file, sep='\t', header=False, index=False)
    print('num of nodes:', len(node_list))
    print(node_list.head())
    
    #写入edgelist
    edge_list['node1'] = ppi_graph[1].map(lambda x: node_list[node_list['gene_name']== x]['id'].values[0])
    edge_list['node2'] = ppi_graph[2].map(lambda x:  node_list[node_list['gene_name']== x]['id'].values[0])
    edge_list.to_csv(edges_list_file, header=None, index=None, sep='\t')
    print('num of edges:', len(edge_list))
    print(all_edge_list)
else:
    node_list = pd.read_csv(node_list_file, header=None, sep='\t')
    print('num of nodes:', len(node_list))
    edge_list = pd.read_csv(edge_list_file, header=None, sep='\t')
    print('num of edges:', len(edge_list))

# all nodes label

num of nodes: 20439
num of edges: 901609


In [4]:
print(node_list.head())
print(edge_list.head())

   0        1
0  0     AKT3
1  1     CDH2
2  2     MED6
3  3    NR2E3
4  4  NAALAD2
   0     1
0  0   877
1  0  1110
2  0  2230
3  0  3004
4  0  3378


###  show ppi network

## 得到一步作用网络的所有nodelist，edgelist，nodelable

In [101]:
# 首先获取autoseed 输出得到的疾病种子
if not os.path.exists(disease_seed_list_file):
    disease_seed = pd.read_csv(seed_input_file, header=None)
    print('num of auto seed output:', len(disease_seed))

    # 映射到ppi网络中，得到seed基因相应id
    disease_seed_list = node_list[node_list[1].isin(disease_seed[0])].reset_index(drop=True)
    print('num of seed in ppi', len(disease_seed_list))
    disease_seed_list.to_csv(disease_seed_list_file, sep='\t', index=False, header=False) 
    disease_edge_list= edge_list[edge_list[0].isin(disease_seed_list[0]) | edge_list[1].isin(disease_seed_list[1])]

    # 写入disease edgelist
    print('num of disease edges', len(disease_edge_list))
    disease_edge_list.to_csv(disease_edge_list_file, sep='\t', header=False, index=False)

    # 写入带lable的nodelist
    disease_node_list = pd.DataFrame()
    disease_node_list['id'] =  pd.concat([disease_edge_list[0], disease_edge_list[1]]).drop_duplicates().reset_index(drop=True)
    disease_node_list['name'] = disease_node_list['id'].map(lambda x: node_list[node_list[0] == x].values[0][1])
    disease_node_list['label'] = disease_node_list['id'].map(lambda x: 'seed' if x in disease_seed_list[0].values else 'unknown')
    print(disease_node_list.head())
    disease_node_list.to_csv(disease_node_list_file, header=False, index=False, sep='\t')
    print('num of disease nodes', len(disease_node_list))
else:
    disease_seed_list = pd.read_csv(disease_seed_list_file, header=None, encoding='utf-8')
    disease_edge_list = pd.read_csv(disease_edge_list_file, header=None, encoding='utf-8')
    disease_node_list = pd.read_csv(disease_node_list_file, header=None, encoding='utf-8')

num of auto seed output: 174
num of seed in ppi 164
num of disease edges 16817
6337
6337
     id    name label
0    12    SRA1  seed
1   437    NEBL  seed
2   489  TXNRD2  seed
3   515   RBCK1  seed
4  1000   CHRM2  seed
num of disease nodes 6337


# 生成疾病的Embedding

In [6]:
embedding_methods = ['Laplacian', 'SVD', 'GF', 'HOPE', 'GraRep', 'DeepWalk', 'node2vec', 'struc2vec', 
              'DeepWalk', 'LINE', 'SDNE', 'GAE']
dim = 512
embedding_output_dir = 'embeddings/' + disease_name + '/'
if not os.path.exists(embedding_output_dir):
    os.makedirs(embedding_output_dir)

# 调用shell 命令生成embedding 文件
for method in embedding_methods:
    embedding_output_file = embedding_output_dir + method + '_' + str(dim) + '.txt'
    if not os.path.exists(embedding_output_file):
        !bionev --input $disease_edge_list_file --output $embedding_output_file --method $method --dimensions $dim

unable to import 'smart_open.gcs', disabling that module
######################################################################
Embedding Method: Laplacian, Evaluation Task: none
######################################################################
Loading training graph for learning embedding...
Graph Loaded...
begin norm_lap_mat
finish norm_lap_mat
finish getLap...
finish eigh(lap_mat)...
Saving embeddings...
Embedding Learning Time: 64.62 s
unable to import 'smart_open.gcs', disabling that module
######################################################################
Embedding Method: SVD, Evaluation Task: none
######################################################################
Embedding Learning Time: 118.21 s
unable to import 'smart_open.gcs', disabling that module
######################################################################
Embedding Method: GF, Evaluation Task: none
######################################################################
Loading training graph for lea




epoch:0 sum of loss:101.99872189760208
epoch:1 sum of loss:86.50332820415497
epoch:2 sum of loss:77.3658752143383
epoch:3 sum of loss:67.3348576426506
epoch:4 sum of loss:55.05392409861088
Saving embeddings...
Embedding Learning Time: 106.15 s
unable to import 'smart_open.gcs', disabling that module
######################################################################
Embedding Method: SDNE, Evaluation Task: none
######################################################################
Loading training graph for learning embedding...
Graph Loaded...

2020-06-10 23:56:05.698590: I tensorflow/core/platform/cpu_feature_guard.cc:142] Your CPU supports instructions that this TensorFlow binary was not compiled to use: AVX2 FMA
2020-06-10 23:56:05.729014: I tensorflow/core/platform/profile_utils/cpu_utils.cc:94] CPU Frequency: 1999965000 Hz
2020-06-10 23:56:05.735029: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5654b2bfe670 initialized for platform Host (this does not gua

# 使用PU-learning 方法对生成Embedding进行评估

In [102]:
# %load pu_learning.py
import numpy as np
import logging
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.naive_bayes import BernoulliNB, GaussianNB
from sklearn.svm import SVC, SVR
from collections import Counter
import pandas as pd

__author__ = 'liuqinyi'
__version__ = '0.1'

logger = logging.getLogger(__name__)


class spies:
    """
    PU spies method, based on Liu, Bing, et al. "Partially supervised classification of
    text documents." ICML. Vol. 2. 2002.
    """
    def __init__(self, first_model, second_model):
        """
        Any two models which have methods fit, predict and predict_proba can be passed,
        for example" `spies(XGBClassifier(), XGBClassifier())`
        """
        self.first_model = first_model
        self.second_model = second_model
        
    def fit(self, X, y,  spie_rate, iterate, oneline, writer):
        """
        Trains models using spies method using training set (X, y).

        Parameters
        ----------
        X : {array-like} of shape = [n_samples, n_features]
            The training input samples.

        y : array-like, shape = [n_samples]
            The target values (1 for positive, 0 for unlabeled).
            
        spie_rate : {float} = 0.2 (default)
            Determines percentage of spies which will be included when training first model.
            
        spie_tolerance : {float} = 0.05 (default)
            Determines tolerated percentage of spies which can come from the first model.
            Using this tolerance threshold is chosen which splits dataset into Likely negative
            and unlabeled groups.

        Returns
        -------
        self : object
            Returns self.
        """
        np.random.seed(42)
        # 循环100次，每次使用随机的S集, 获取绝对负样
        P = X[y[0] == 1]
        U = X[y[0] == 0]
        RN_id = []
        for i in range(iterate):
            # 随机获取间谍集S
            spie_mask = np.random.random(y[0].sum()) < spie_rate
            S = P[spie_mask]
            # U+S
            US = np.vstack([U, P[spie_mask]])
            US_y = np.hstack([np.zeros((y[0] == 0).sum()), np.ones(spie_mask.sum())])
            # 为了后面更好地统计每次循环中的绝对负样，这里在y中加入一列对照id
            # US_id = np.hstack([y[1][y[0] == 0], y[1][y[0] == 1][spie_mask]])

            # P-S
            PS = P[~spie_mask]
            # print('num of P-S:', PS.shape[0], 'num of U+S:', US.shape[0])

            # 准备第一个模型的训练集
            USP = np.vstack([US, PS])
            USP_y = np.hstack([np.zeros(US.shape[0]), np.ones(PS.shape[0])])

            # Fit first model
            self.first_model.fit(X=USP, y=USP_y)

            # 确认取绝对负值的阈值tr, 由S集预测概率的10%的分为数决定
            S_prob = self.first_model.predict_proba(S)
            S_prob = S_prob[:, 1]
            tr = np.percentile(S_prob, 10)

            U_prob = self.first_model.predict_proba(U)
            U_prob = U_prob[:, 1]
            # 得到一次循环的负样的ID
            N_id = y[1][y[0] == 0][U_prob <= tr]
            RN_id = RN_id + N_id.tolist()

        # 统计在iter次循环中都出现的负样即为绝对负样
        RN_dict = Counter(RN_id)
        RN_id_list = []
        for (K, V) in RN_dict.items():
            if V == iterate:
                RN_id_list.append(K)
        RN_id_list = np.array(RN_id_list)
        
        # 得到绝对负样的特征集
        RN = X[(y[0] == 0) & (np.isin(y[1], RN_id_list))]
        RNP = np.vstack([RN, P])
        RNP_y = np.hstack([np.zeros(RN.shape[0]), np.ones(P.shape[0])])
        print('正样数:', P.shape[0], '绝对负样数:', RN.shape[0])

        # Fit second model
        X_train, X_test, y_train, y_test = train_test_split(RNP, RNP_y, test_size=0.2)
        self.second_model.fit(X_train, y_train)
        score = self.second_model.score(X_test, y_test)
        print('score:', score)

        # 交叉验证
        cross_scores = cross_val_score(self.second_model, RNP, RNP_y, cv=5).mean()
        print('cross scores:', cross_scores)
        y_predict = self.second_model.predict(RNP)
        cf_mx = confusion_matrix(y_true=RNP_y, y_pred=y_predict)
        print('confusion matrix:', cf_mx)
        oneline.append('%d/%d'%(cf_mx[1][1], cf_mx[1].sum()))
        oneline.append('%d/%d'%(cf_mx[0][0], cf_mx[0].sum()))
        
        # 计算最后一次S被判定正样的结果
        S_predict = self.second_model.predict(S)
        print('Predict of S:%d/%d' % (S_predict.sum(), S.shape[0]))
        oneline.append('%d/%d' % (S_predict.sum(), S.shape[0]))
        oneline.append(score)
        oneline.append(cross_scores)
        
        # Unknown 预测结果
        unknown = X[(y[0] == 0) & (np.isin(y[1], RN_id_list, invert=True))]
        unknown_predict = self.second_model.predict(unknown)
        print('Predict of Real Unknown:%d/%d' % (unknown_predict.sum(), unknown.shape[0]))
        oneline.append('%d/%d' % (unknown_predict.sum(), unknown.shape[0]))
        writer.writerow(oneline)

    def predict(self, X):
        """
        Predicts classes for X. Uses second trained model from self.

        Parameters
        ----------
        X : {array-like} of shape = [n_samples, n_features]
            The training input samples.

        Returns
        -------
        y : array of shape = [n_samples]
            The predicted classes.
        """
        return self.second_model.predict(np.array(X))
    
    def predict_proba(self, X):
        """
        Predict class probabilities for X. Uses second trained model from self.

        Parameters
        ----------
        X : {array-like} of shape = [n_samples, n_features]
            The training input samples.

        Returns
        -------
        p : array of shape = [n_samples, n_classes]
            The class probabilities of the input samples.
        """
        return self.second_model.predict_proba(np.array(X))[:,1]


def load_disease_embedding(embedding_file_name):
    with open(embedding_file_name) as f:
        node_num, emb_size = f.readline().split()
        print('Nodes with embedding: %s' % node_num)
        embedding_look_up = {}
        for line in f:
            vec = line.strip().split()
            node_id = vec[0]
            emb = [float(x) for x in vec[1:]]
            emb = emb / np.linalg.norm(emb)
            emb[np.isnan(emb)] = 0
            embedding_look_up[node_id] = np.array(emb)
    f.close()
    return embedding_look_up


def main(embedding_file_name, disease_label_file, oneline, writer):
    
    # 读取label 文件，两列(id , label("seed", "unknown"))
    disease_id_label = pd.read_csv(disease_label_file, sep='\t', header=None)

    # 读取embedding文件
    # output_file = disease_name + disease_label_file.split('/')[-1]
    embedding_look_up = load_disease_embedding(embedding_file_name)

    # 获得X(shape=[n_sample, n_features]) Y(shape=[label("0","1"), id])
    X = np.array([embedding_look_up[str(inx)] for inx in disease_id_label[0]])
    Y = disease_id_label[2].map(lambda x: 1 if x == 'seed' else 0).values
    ID = disease_id_label[0].values
    Y = np.stack([Y, ID], axis=0)
    print('X.shape:', X.shape, 'Y.shape:', Y.shape)
    oneline.append(X.shape[0])
    # 训练Pulearning 对象，这里使用spise方法，后续可以加入其它两种方法
    model = spies(GaussianNB(), SVC(kernel='linear', gamma='auto',  probability=True))
    model.fit(X, Y, 0.2, 100, oneline, writer)
    df = pd.DataFrame()
    df['id'] =  disease_id_label[0]
    df['name'] = disease_id_label[1]
    df['prob'] = pd.DataFrame(model.predict_proba(X))
    save_file = 'output/result/' + oneline[0] + '_' + oneline[1] + '_' + oneline[2]+'_predict.csv'
    df.to_csv(save_file, index=False)

if __name__ == '__main__':
    output_dir='output'
    if not os.path.exists(output_dir):
        os.makedirs(output_dir + '/eval')
        os.makedirs(output_dir + '/result')
    eval_file =output_dir + '/eval/' + disease_name + '.csv' 
    
    eval_file_header = ['疾病', 'Embedding方法', '维度', '一步作用网络获取基因数', '正样（预测/总数）', '100次循环负样（预测/总数）', 'S集（预测/最后一次迭代）', 'Score', 'Cross_Score', 'Unknown（预测/总数）']
    with open(eval_file, 'w', newline='', encoding='utf-8') as ef:
        writer = csv.writer(ef, delimiter=',')
        writer.writerow(eval_file_header)
        for input_embedding_file in glob.glob(embedding_output_dir + '*'):
            print(input_embedding_file)
            try:
                oneline = []
                embedding_file = input_embedding_file.split('/')[-1]
                method, dim = embedding_file.split('_')
                dim = dim.split('.')[0]
                oneline = [disease_name, method, dim]
                main(input_embedding_file, disease_node_list_file, oneline, writer)
            except Exception as e:
                print('%s has happend a exception' % method)
                print(e)

embeddings/扩张性心肌病/node2vec_256.txt
Nodes with embedding: 6337
X.shape: (6337, 256) Y.shape: (2, 6337)
正样数: 161 绝对负样数: 1121
score: 0.9922178988326849
cross scores: 0.9851805690661479
confusion matrix: [[1121    0]
 [   9  152]]
Predict of S:24/27
Predict of Real Unknown:3644/5055
embeddings/扩张性心肌病/HOPE_128.txt
Nodes with embedding: 6337
X.shape: (6337, 128) Y.shape: (2, 6337)
正样数: 161 绝对负样数: 128
score: 1.0
cross scores: 1.0
confusion matrix: [[128   0]
 [  0 161]]
Predict of S:27/27
Predict of Real Unknown:5912/6048
embeddings/扩张性心肌病/DeepWalk_128.txt
Nodes with embedding: 6337
X.shape: (6337, 128) Y.shape: (2, 6337)
正样数: 161 绝对负样数: 1602
score: 0.9915014164305949
cross scores: 0.9903634432140098
confusion matrix: [[1602    0]
 [   7  154]]
Predict of S:24/27
Predict of Real Unknown:3436/4574
embeddings/扩张性心肌病/Laplacian_128.txt
Nodes with embedding: 6337
X.shape: (6337, 128) Y.shape: (2, 6337)
正样数: 161 绝对负样数: 0
Laplacian has happend a exception
The number of classes has to be greater than

/home/liuqinyi/anaconda3/envs/bionev/lib/python3.7/site-packages/sklearn/model_selection/_split.py:667: UserWarning: The least populated class in y has only 2 members, which is less than n_splits=5.
  % (min_groups, self.n_splits)), UserWarning)


embeddings/扩张性心肌病/GraRep_128.txt
Nodes with embedding: 6337
X.shape: (6337, 128) Y.shape: (2, 6337)
正样数: 161 绝对负样数: 1914
score: 0.9975903614457832
cross scores: 0.993734939759036
confusion matrix: [[1914    0]
 [   3  158]]
Predict of S:26/27
Predict of Real Unknown:1904/4262
embeddings/扩张性心肌病/node2vec_128.txt
Nodes with embedding: 6337
X.shape: (6337, 128) Y.shape: (2, 6337)
正样数: 161 绝对负样数: 1495
score: 0.9909638554216867
cross scores: 0.9836930804790157
confusion matrix: [[1495    0]
 [  11  150]]
Predict of S:24/27
Predict of Real Unknown:3344/4681
embeddings/扩张性心肌病/GF_256.txt
Nodes with embedding: 6337
X.shape: (6337, 256) Y.shape: (2, 6337)
正样数: 161 绝对负样数: 1954
score: 0.9905437352245863
cross scores: 0.9929078014184396
confusion matrix: [[1954    0]
 [   9  152]]
Predict of S:23/27
Predict of Real Unknown:3596/4222
embeddings/扩张性心肌病/struc2vec_256.txt
Nodes with embedding: 6337
X.shape: (6337, 256) Y.shape: (2, 6337)
正样数: 161 绝对负样数: 4215
score: 0.997716894977169
cross scores: 0.9974

# 使用训练好的模型预测疾病相关基因